In [2]:
import requests
import json
from bs4 import BeautifulSoup as bs
from time import sleep as slp
from typing import List
import random
import pandas as pd
import re
from datetime import datetime
import os

In [3]:

config = {"User-Agent": 
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.54",
            } #伪装成浏览器访问
# 第一步：先爬出5000个新闻链接
# 滚动网https://news.sina.com.cn/roll/#pageid=153&lid=2515&k=&num=50&page={} 可以查到
# 滚动网链接来源https://feed.mix.sina.com.cn/api/roll/get?pageid=153&lid=2515&k=&num=50&page={} 可以用network查出来
# 经过我人工测试下来，可以采取一次拉50一共拉50次的操作 （然后这个破网站就会跳NaN） 所以这里有2500条
def get_urls(urlx, start_page, dt) -> List[str]: #写一个爬其中一页的函数
    dt_c = datetime.strptime(dt, '%Y-%m-%d')
    dt_stamp = int(dt_c.timestamp())
    #print(dt_stamp)
    st_stamp = dt_stamp + 86400
    news_list = []
    try:
        repo = requests.get(url = urlx, params = {"pageid": 153, "lid": 2515, "num": 50, "date": dt, "page": start_page, "stime": st_stamp, "ctime": st_stamp, "etime": dt_stamp}, headers = config)
        repo.encoding = "UTF-8"
        tex = repo.json()
        #然后直接找链接
        leng = len(tex["result"]["data"])
        news_list += [ tex["result"]["data"][index]["url"] for index in range(leng)] 
    except:
        news_list.append["cannot reach"]
    return news_list



dates = pd.date_range(start = "2019-01-01", periods = 360).strftime("%Y-%m-%d").tolist()
dates = dates[::-1]
for dt in dates:
    for page in range(1, 10):
        lst = get_urls(f"https://feed.mix.sina.com.cn/api/roll/get", page, dt)
        with open(r'./news_links_lasty.txt', 'a', encoding = "utf-8") as f:
            for item in lst:
                if item != "cannot reach":
                    f.write(f"{item}\n")
            f.close()

In [4]:
#第二步：处理新闻要素
from PIL import Image
import traceback
import multiprocessing
config = {"User-Agent": 
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.54",
        }

def get_html(url):
    slp(random.uniform(0.3, 0.5))
    repo = requests.get(url = url, headers = config)
    repo.encoding = 'utf-8'
    html = repo.text
    # print(url)
    return html

def parse_html(html, url):
    try:
        soup = bs(html, "lxml") # 先解析网页
        
        # 新浪有个贼坑的专栏叫"创事记"，如果作者是他的话发行日期和文章截取得特判
        author = re.findall('<meta name="mediaid" content="(.*?)"', html)[0] # 作者
        if author == "创事记":
            publish_time = soup.find("span", id = "pub_date").text # 发行日期
            publish_time = publish_time.strip()
            date_str = publish_time
            date_obj = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
        else:
            publish_time = soup.find("span", class_ = "date").text # 发行日期
            date_str = publish_time
            date_obj = datetime.strptime(date_str, "%Y年%m月%d日 %H:%M")
        
        new_date_str = date_obj.strftime("%Y-%m-%d")
        publish_time = date_obj.strftime("%Y-%m-%d %H:%M")
        dir_name = re.findall('[0-9]\/(.+?)\.shtml', url)[0] # 给网站起个名字
        dir_name = './websites_ten/' + new_date_str + dir_name
        # print(dir_name)
        try:
            if not os.path.exists(dir_name):
                os.mkdir(dir_name) # 一个网站一个文件夹
                
        except:
            print("website already create")
        
        dir_img = dir_name + '/images'
        
        try:
            if not os.path.exists(dir_img):
                os.mkdir(dir_img) # 一个网站一个文件夹
                
        except:
            print("image site already create")
        
        title = re.findall('<meta property="og:title" content="(.*?)"', html)[0] # 新闻标题
        
        description = re.findall('<meta name="description" content="(.*?)"', html) # 摘要
        if len(description) > 0:
            description = description[0]
        keywords_source = soup.find("div", class_ = "keywords") # 关键词-1
        keywords = []
        if keywords_source is not None:
            keywords = keywords_source.find_all("a") # 关键词-2
            keywords = [item.text for item in keywords] # 关键词-3
        # print(keywords)
        passage_naked = "" # 整个文章
        passage_origin = ""
        passage = soup.select(".article p") # 直接元素选择器
        if passage == []:
            passage = soup.select("#artibody p") # 直接元素选择器
        for content in passage:
            passage_naked += (content.text + "\n")
            passage_origin += content.text
        # print(passage_naked)
        dic_ = {"url": url, "title": title, "author": author, "description": description, "publish_time": publish_time, "keywords": keywords}
        json_dic = json.dumps(dic_, indent = 2) # 将以上文件通过dict转化为json utf-8
        # print(json_dic)
        json_dir = dir_name + '/config.json' # json文件的地址
        article_dir = dir_name + '/article.txt' # 文章内容地址
        article_without = dir_name + '/article_origin.txt' # 文章内容地址
        with open(json_dir, 'w') as f:
            f.write(json_dic) # 写入config.json文件
            f.close()
        
        with open(article_dir, 'w') as f:
            f.write(passage_naked) # 写入article.txt文件
            f.close()
            
        with open(article_without, 'w') as f:
            f.write(passage_origin) # 写入article.txt文件
            f.close()
            
        img_links = []
        imgs = soup.find_all("div", class_ = "img_wrapper")
        for item in imgs:
            img_link_origin = item.find("img")
            #print(img_link_origin)
            try:
                img_link_origin = img_link_origin["src"]
                if not(re.match("^https:", img_link_origin) or re.match("^http:", img_link_origin)):
                    img_link_origin = "https:" + img_link_origin
                if img_link_origin is not None:
                    img_links.append(img_link_origin)
            except:
                pass
        # print(img_links)
        index = 0
        img_dic = {"empty": 1} #图片对应的id和链接
        
        datax = {}
        for one in img_links:
            img_dic["empty"] = 0;
            index += 1
            name_tmp = one.split('/')[-1]
            datax[f"{index}"] = [name_tmp, one];
            # img_data = requests.get(url = one).content
            # with open(os.path.join(dir_img, name_tmp), 'wb') as f:
            #     f.write(img_data)
            #     f.close()
        
        img_dic['data'] = datax
        img_json_dic = json.dumps(img_dic, indent = 2)
        json_dir = dir_img + '/img_config.json' # json文件的地址
        with open(json_dir, 'w') as f:
            f.write(img_json_dic) # 写入img_config.json文件
            f.close()
    except Exception as ex:
        print(url)

    
    
def main():
    # with multiprocessing.Pool(processes=8) as pool:
    with open(r"./news_links_lasty.txt", "r", encoding = "utf-8") as f:
        urlsx = f.readlines()
    urlsx = [ url.strip() for url in urlsx ]
    for url in urlsx:
        parse_html(get_html(url), url)
        
        
if __name__ == "__main__":  
   main() 

https://tech.sina.com.cn/it/2019-11-11/doc-iicezuev8570646.shtml
https://tech.sina.com.cn/it/2019-11-08/doc-iicezuev7953475.shtml
https://tech.sina.com.cn/i/2019-11-08/doc-iicezzrr7979660.shtml
https://tech.sina.com.cn/it/2019-11-07/doc-iicezuev7760540.shtml
https://tech.sina.com.cn/it/2019-11-06/doc-iicezzrr7510284.shtml
https://tech.sina.com.cn/it/2019-11-05/doc-iicezuev7237530.shtml
https://tech.sina.com.cn/t/2019-11-04/doc-iicezuev7150354.shtml
https://tech.sina.com.cn/it/2019-11-04/doc-iicezzrr7185005.shtml
https://tech.sina.com.cn/it/2019-11-04/doc-iicezuev6967180.shtml
https://tech.sina.com.cn/t/2019-10-31/doc-iicezuev6318884.shtml
https://tech.sina.com.cn/it/2019-10-31/doc-iicezuev6074751.shtml
https://tech.sina.com.cn/it/2019-10-30/doc-iicezuev5977618.shtml
https://tech.sina.com.cn/it/2019-10-30/doc-iicezuev5830573.shtml
https://tech.sina.com.cn/it/2019-10-29/doc-iicezzrr5626571.shtml
https://tech.sina.com.cn/mobile/n/n/2019-10-29/doc-iicezzrr5591327.shtml
https://tech.sina.co

In [5]:
#以下是专门为新浪的苹果新闻定制的小爬虫
from PIL import Image
config = {"User-Agent": 
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.54",
        }
with open(r"./pa2.txt", "r", encoding = "utf-8") as f:
    urlsx = f.readlines()
# print(urlsx)
urlsx = [ url.strip() for url in urlsx ]
# print(urlsx)
for url in urlsx:
    try:
        slp(random.uniform(0.3, 0.5))
        repo = requests.get(url = url, headers = config)
        repo.encoding = 'utf-8'
        html = repo.text
        # print(html)
        soup = bs(html, "lxml") # 先解析网页
        
        #关键词模块重做：根据观察，放在meta里面了
        keywords = re.findall(r'<meta name="keywords" content="(.*?)"' , html)[0]
        keywords = keywords.split(',')
        # print(keywords)
        
        title = re.findall('<meta property="og:title" content="(.*?)"', html)[0] # 新闻标题
        author = re.findall('<meta property="article:author" content="(.*?)"', html)[0] # 作者
        # print(author)
        
        #发行日期模块重做
        publish_time = re.findall(r'<span id="pub_date">(.*?)</span>', html)[0] # 发行日期
        date_obj = datetime.strptime(publish_time, "%Y年%m月%d日%H:%M")
        new_date_str = date_obj.strftime("%Y-%m-%d")
        publish_time = date_obj.strftime("%Y-%m-%d %H:%M")
        # print(new_date_str)
        dir_name = re.findall('[0-9]\/(.+?)\.shtml', url)[0] # 给网站起个名字
        dir_name = './websites_ten/' + new_date_str + dir_name
        # print(dir_name)
        try:
            if not os.path.exists(dir_name):
                os.mkdir(dir_name) # 一个网站一个文件夹
                dir_img = dir_name + '/images'
                os.mkdir(dir_img) # 文件夹中有一个图片夹
        except:
            print("website already create")

        description = re.findall('<meta name="description" content="(.*?)"', html) # 摘要
        if len(description) > 0:
            description = description[0]
        # print(keywords)
        passage_naked = "" # 整个文章
        passage = soup.select("#artibody p") # 直接元素选择器
        for content in passage:
            passage_naked += (content.text + "\n")
        # print(passage_naked)
        dic_ = {"url": url, "title": title, "author": author, "description": description, "publish_time": publish_time, "keywords": keywords}
        json_dic = json.dumps(dic_, indent = 2) # 将以上文件通过dict转化为json utf-8
        # print(json_dic)
        json_dir = dir_name + '/config.json' # json文件的地址
        article_dir = dir_name + '/article.txt' # 文章内容地址
        with open(json_dir, 'w') as f:
            f.write(json_dic) # 写入config.json文件
            f.close()
        
        with open(article_dir, 'w') as f:
            f.write(passage_naked) # 写入article.txt文件
            f.close()
            
        img_links = []
        imgs = soup.find_all("div", class_ = "img_wrapper")
        for item in imgs:
            img_link_origin = item.find("img")["src"]
            if not re.match("^https:", img_link_origin):
                img_link_origin = "https:" + img_link_origin
            img_links.append(img_link_origin) 
        # print(img_links)
        index = 0
        img_dic = {"empty": 1} #图片对应的id和链接
        
        for one in img_links:
            img_dic["empty"] = 0;
            index += 1
            name_tmp = one.split('/')[-1]
            img_dic[f"{index}"] = name_tmp;
            # img_data = requests.get(url = one).content
            # with open(os.path.join(dir_img, name_tmp), 'wb') as f:
            #     f.write(img_data)
            #     f.close()
                
        img_json_dic = json.dumps(img_dic, indent = 2)
        json_dir = dir_img + '/img_config.json' # json文件的地址
        with open(json_dir, 'w') as f:
            f.write(img_json_dic) # 写入img_config.json文件
            f.close()
    except Exception as ex:
        print(url)
        print(repo)

http://tech.sina.cn/csj/2019-07-10/doc-ihytcerm2496466.d.html
<Response [200]>


In [32]:
path = r'D:\involuntary\works\missing-semester-of-python\part1\websites'
files = os.listdir(path)

for file in files:
    try:
        file_path = os.path.join(path, file + '\images\img_config.json')
        with open(file_path, 'r') as f:
            pa = json.load(f)
            if 'data' in pa.keys():
                continue
            new_pa = {}
            data = {}
            for key in pa.keys():
                if key == 'empty':
                    new_pa[key] = pa[key]
                    if new_pa[key] == 1:
                        continue
                else:
                    data[key] = pa[key]
            new_pa['data'] = data
            xp = json.dumps(new_pa, indent = 2)
            
            
        with open(file_path, 'w') as f:
            f.write(xp)
    except:
        print(traceback.print_exc())

In [12]:
import os

def remove_indent_from_line(line):
    return line.lstrip('　 ')

def process_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    modified_lines = [remove_indent_from_line(line) if line.startswith('　　') or line.startswith(' ') else line for line in lines]

    with open(file_path, 'w') as file:
        file.writelines(modified_lines)

# Replace 'directory_path' with the path to the directory containing your txt files
directory_path = r'D:\involuntary\works\missing-semester-of-python\part1\websites'



for file_name in os.listdir(directory_path):
    x = directory_path + '\\' + file_name
    print(x)
    for file_n in os.listdir(x):
        print(file_n)
        if file_n == 'article.txt':
            file_path = os.path.join(x, file_n)
            process_file(file_path)


D:\involuntary\works\missing-semester-of-python\part1\websites\2023-04-01doc-imynwriw7099269
article.txt
article_origin.txt
config.json
images
D:\involuntary\works\missing-semester-of-python\part1\websites\2023-04-01doc-imynwriy3876374
article.txt
article_origin.txt
config.json
images
D:\involuntary\works\missing-semester-of-python\part1\websites\2023-04-01doc-imynwvrx5442728
article.txt
article_origin.txt
config.json
images
D:\involuntary\works\missing-semester-of-python\part1\websites\2023-04-01doc-imyperrm3094050
article.txt
article_origin.txt
config.json
images
D:\involuntary\works\missing-semester-of-python\part1\websites\2023-04-02doc-imyntrfm7242443
article.txt
article_origin.txt
config.json
images
D:\involuntary\works\missing-semester-of-python\part1\websites\2023-04-02doc-imynyyrf0417580
article.txt
article_origin.txt
config.json
images
D:\involuntary\works\missing-semester-of-python\part1\websites\2023-04-02doc-imynyyrf0419315
article.txt
article_origin.txt
config.json
images

In [18]:
import json
import selenium
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.chrome.options import Options as ChromeDriverOpt

In [12]:
#假chrome
virtual_driver = selenium.webdriver.Chrome(service = ChromeService(ChromeDriverManager().install(), options = ChromeDriverOpt()))
virtual_driver.get(f"https://new.qq.com/ch/tech/")

news_lists = []
while True:
    #模拟滚动到底 能扒拉多少扒拉多少
    virtual_driver.execute_script("window.scrollBy(0, -10)") #往回倒一点点
    virtual_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #往下倒一个整屏
    slp(2) #等两秒让他加载一会
    